In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, parse_qs

In [2]:
base_url = 'https://sso.uni-muenster.de/LearnWeb/learnweb2'
search_url_pattern = 'https://sso.uni-muenster.de/LearnWeb/learnweb2/course/search.php?search={search_string}'

In [64]:
payload = {'httpd_username': '', 'httpd_password': ''}
        
try:
    session = requests.Session()
    session_response = session.post(base_url, data=payload)
    print(session_response.status_code)
    if session_response.status_code == 401:
        print('Test')
        SystemExit("401 unauthorized")
    
except requests.exceptions.RequestException as e:
    raise SystemExit(e)

session_header = session_response.headers

session_id = session_header['set-Cookie']

headers = {'Cookie': session_id}

401
Test


In [7]:
def response_to_soup(res: requests.Response) -> BeautifulSoup:
    document: str = res.text
    return BeautifulSoup(document, "lxml")


def url_to_soup(url: str, headers: dict = None) -> BeautifulSoup:
    return response_to_soup(requests.get(url, headers=headers))

In [15]:
soup = url_to_soup(base_url, headers)
        
current_semester = soup.find('ul', attrs={'class':'sub-sub-menu'})

if current_semester is None:
    SystemExit("could not find element")

courses = []
course_list_elements = current_semester.find_all('li', attrs={'class':'sub-sub-menu-item' })

if course_list_elements is not None:
    for i in current_semester.find_all('li', attrs={'class':'sub-sub-menu-item' }):
        courses.append(i.find('span').getText())
else:
    SystemExit("could not find element")

In [54]:
for course in courses:
    search_url = search_url_pattern.format(search_string=requests.utils.quote(course))
    
    soup = url_to_soup(search_url, headers)
    
    number_search_results_string = soup.find('h2').getText()
    
    if number_search_results_string is not None:
        number_search_results = int(re.sub(r'[^0-9,.]', ' ', number_search_results_string))
    else:
        SystemExit("could not find element")
    
    if number_search_results > 1:
        SyntaxWarning("Learnweb Search results in more than 1 courses")
    elif number_search_results == 0:
        SystemError("Learnweb Search results in 0 courses")
        
    course_name = soup.find('a', attrs={'class':'aalink'}).text
        
    qis_url = soup.find('div', attrs={'class':'content'}).find('div', attrs={'class':'summary'}).find('a')
    
    if qis_url is not None:
        qis_url = qis_url['href']+'&language=en'
        
        parsed_qis_url = urlparse(qis_url)
        course_id = parse_qs(parsed_qis_url.query)['publishid'][0]
        
        soup = url_to_soup(qis_url)
        
        # Quispos table crawling
        table_data = []
        table = soup.find('table', attrs={'summary':'Übersicht über alle Veranstaltungstermine'})
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            filtered_cols = []
            for ele in cols:
                room = ele.find('a', attrs={'class':'regular'})

                if room is not None:
                    room_link = room['href']

                    parsed_room_link = urlparse(room_link)
                    link_params = parse_qs(parsed_room_link.query)

                    if 'raum.rgid' in link_params:
                        room_id = link_params['raum.rgid'][0]
                        filtered_cols.append(room_id)
                    else:
                        filtered_cols.append(ele.text.strip())
                else:
                    filtered_cols.append(ele.text.strip())
                
            table_data.append(filtered_cols)

None
None
None
None
None
<a class="regular" href="https://studium.uni-muenster.de/qisserver/rds?state=verpublish&amp;status=init&amp;vmfile=no&amp;moduleCall=webInfo&amp;publishConfFile=webInfoRaum&amp;publishSubDir=raum&amp;keep=y&amp;raum.rgid=17157" title="See details on Room LEO 18.3">Leonardo-Campus 18 - LEO 18.3</a>
None
None
None
None
None
None
None
None
None
None
None
<a class="regular" href="https://studium.uni-muenster.de/qisserver/rds?state=verpublish&amp;status=init&amp;vmfile=no&amp;moduleCall=webInfo&amp;publishConfFile=webInfoRaum&amp;publishSubDir=raum&amp;keep=y&amp;raum.rgid=17157" title="See details on Room LEO 18.3">Leonardo-Campus 18 - LEO 18.3</a>
None
None
None
None
None
None
None
None
None
None
None
<a class="regular" href="https://studium.uni-muenster.de/qisserver/rds?state=verpublish&amp;status=init&amp;vmfile=no&amp;moduleCall=webInfo&amp;publishConfFile=webInfoRaum&amp;publishSubDir=raum&amp;keep=y&amp;raum.rgid=17157" title="See details on Room LEO 18.3">Le

In [63]:
for entry in table_data[1:]:
    weekday = entry[1]
    start_end_time = entry[2].replace(u'\xa0', u' ')
    start_time, end_time = start_end_time.split(' to ')
    rythm = entry[3]
    start_end_date = entry[4]
    room_id = entry[5]

## Get Rooms

In [17]:
document = url_to_soup('https://studium.uni-muenster.de/qisserver/rds?state=change&type=6&moduleParameter=raumSelect&nextdir=change&next=SearchSelect.vm&target=raumSearch&subdir=raum&source=state%3Dchange%26type%3D5%26moduleParameter%3DraumSearch%26nextdir%3Dchange%26next%3Dsearch.vm%26subdir%3Draum%26_form%3Ddisplay%26topitem%3Dfacilities%26subitem%3DsearchFacilities%26function%3Dnologgedin%26field%3Ddtxt&targetfield=dtxt&_form=display&noDBAction=y&init=y')

In [35]:
fieldset = document.find_all('fieldset')[1]
room_ids = []
for room in fieldset.find_all('a', href=True):
    room_link = room['href']
    
    parsed_room_link = urlparse(room_link)
    link_params = parse_qs(parsed_room_link.query)
    if 'raum.rgid' in link_params:
        room_id = link_params['raum.rgid'][0]
        room_ids.append(room_id)

In [47]:
url_pattern = 'https://studium.uni-muenster.de/qisserver/rds?state=wsearchv&search=3&alias_einrichtung.eid=einrichtung.dtxt&alias_k_raumart.raumartid=k_raumart.dtxt&raum.rgid={room_id}&P_start=0&P_anzahl=10&_form=display&language=en'

In [95]:
for room_id in room_ids[0:5]:
    url_pattern_formatted = url_pattern.format(room_id=room_id)
    
    document = url_to_soup(url_pattern_formatted)
    
    info_row = document.find('div', attrs={'class':'InfoLeiste'}).text
    hits = int(re.findall(r'(.*)(?=hits)', info_row)[0])
    
    if hits != 0:
        rows = document.find_all('div', attrs={'class':'erg_list_entry'})
        
        room_name: str = None
        room_address: str = None
        room_seats: int = None
        for row in rows:
            row_label = row.find('div', attrs={'class':'erg_list_label'}).text
            if 'Room:' in row_label:
                room_name = row.find('strong').text.strip().replace(',','')
            elif 'Building Address:' in row_label:
                room_address = row.find('div', attrs={'class':'erg_list_label'}).next_sibling.strip().replace(',','')
            elif 'Equipment:' in row_label:
                room_seats = int(re.findall(r'(?<=Plätze:)(.\s*[0-9]*)(?=[a-zA-Z]*)', row.text)[0])
                
        print(room_id, room_name, room_address, room_seats)


17136 1.05 Alter Steinweg 6/7 50
17262 13 Schlossgarten 4 0
18050 17 Orléans-Ring 10 0


## Get long and lat of room

In [2]:
open_street_map_url = 'https://nominatim.openstreetmap.org/search?q={query}&format=json&polygon=1&addressdetails=1'

In [3]:
open_street_map_url_formatted = open_street_map_url.format(query='Leonardo-Campus 18')

In [8]:
location_data = requests.get(open_street_map_url_formatted).json()

In [13]:
lon: str = None
lat: str = None
for data in location_data:
    
    if 'address' in data:
        address = data['address']
        
        if 'city' in address:
            city = address['city']
            
            if city.casefold() == 'münster':
                
                if 'lat' in data:
                    lat = data['lat']
                if 'lon' in data:
                    lon = data['lon']
                    
                break
            

NameError: name 'city1' is not defined